In [1]:
# Data
import numpy as np
import pandas as pd
excel_file = "data.xlsx"
preference_matrix = pd.read_excel(excel_file)


In [5]:
# Scheme 1- Voting for one (plurality voting) -- [1,0,0,0,...,0] {Sai}
# VOTING PREFERENCE-PLURALITY VOTING
#HELPER FUNCTIONS
def distance(colom,result,desired):
    temp = colom[colom == desired]
    index = temp.index
    winposition_in_true_pref = int(index[0][0])
    return winposition_in_true_pref - 1

#HAPPINESS CALCULATOR FOR PLURALITY
def HappinessCalculator(preference_matrix,winner):
    weight = preference_matrix.shape[0] #ONLY FOR PLURALITY
    d = 0
    global_happiness = 0
    for (indexlabel,column_series) in preference_matrix.iteritems():
        for item in column_series.iteritems():
            if(item[1] == winner):
                global_happiness+=1
                break
            else:
                dist = distance(column_series,item[1],winner)
                weight_voter = weight -dist
                d= 1/(1+dist*weight_voter)
                global_happiness+= d
                break
            weight-=1
        weight = preference_matrix.shape[0]
    print('Happiness of the voters is : ' + str(global_happiness))
    
def UpdateWinnerCheck(matrix,considered,voter,index_of_voter,winner):
    first_pref = matrix.at['1st preference', voter]
    matrix.at[index_of_voter,voter] = first_pref
    matrix.at['1st preference', voter] = considered
    w,x = PluralityWinner(matrix)
    return w,matrix
    
def CompareWinners(preference_matrix, winner, considered_winner):
    count = 0
    s = 0
    for voter in preference_matrix.iteritems():
        voter_data = voter[1]
        temp = voter_data[voter_data == winner]
        index = temp.index
        if(index[0] != "10th preference"):
            win_position = int(index[0][0])
        else:
            win_position = 10
        temp2 = voter_data[voter_data == considered_winner]
        index2 = temp2.index
        if(index2.size == 0):
            print("Something wrong...voter preferences are not unique")
        if(index2[0] != "10th preference"):
            consider_position = int(index2[0][0])
        else:
            consider_position = 10
        if(consider_position < win_position):
            matrix_copy = preference_matrix.copy()
            updated_winner,updated_matrix=UpdateWinnerCheck(matrix_copy,considered_winner,voter[0],index2,winner)
            if(updated_winner != winner):
                print('Tactical voting possible because ' + considered_winner + ' is preferred over ' + winner +' by ' + voter[0])
                count+=1
                s+=1
                print('Updated winner after tactical voting is '+str(updated_winner))
                print('Updated preference list for '+voter[0])
                print(updated_matrix[voter[0]].T)
                HappinessCalculator(updated_matrix,updated_winner)
                print('###################')
    if(count == 0):
            print('Tactical voting not possible for '+considered_winner+' due to Lexographic rule or no voter prefers him over the winner')
            print('###################')
    return s

#DETERMINES WINNER FOR PLURALITY
def PluralityWinner(preference_matrix):
    winner_row = preference_matrix.T.sort_values('1st preference')
    unsorted_winrow=winner_row.sort_values('1st preference')['1st preference'].value_counts()
    sorted_winrow = unsorted_winrow.sort_index()
    winner = sorted_winrow.idxmax()
    return winner, unsorted_winrow

def TacticalVoting(preference_matrix, unsorted_winrow, winner):
    mods = 0
    first_row = preference_matrix.loc['1st preference']
    if(unsorted_winrow[0]>unsorted_winrow[1]+1): #WITHOUT VOTER COLLUSION
        print("Tactical Voting Not Possible In this Case without VOTER COLLUSION")
        return
    print("Tactical voting may be possible if somebody prefers other than the winner and wins with lexographic rule")
    winner_count = unsorted_winrow[0]
    possible_winners = unsorted_winrow[(unsorted_winrow == winner_count - 1) | (unsorted_winrow == winner_count)]
    for item in possible_winners.iteritems():
        if(item[0] == winner):
            #print("No tactical voting for winner as an alternative")
            pass
        else:
            considered_winner = item[0]
            print('Checking if ' + considered_winner + ' is preferred over the winner')
            mods+=CompareWinners(preference_matrix,winner,considered_winner)
            pass
        pass
    return mods



# VOTING PREFERENCE-PLURALITY VOTING
winner, unsorted_winrow = PluralityWinner(preference_matrix)
print('According to plurality voting the winner is ' + winner )
print('###################')
HappinessCalculator(preference_matrix,winner)
print('###################')
mods = TacticalVoting(preference_matrix,unsorted_winrow,winner)
risk = mods/preference_matrix.shape[1]
print('Risk of tactical voting according to plurality voting scheme is '+str(risk))


According to plurality voting the winner is A
###################
Happiness of the voters is : 2.4175647881530233
###################
Tactical voting may be possible if somebody prefers other than the winner and wins with lexographic rule
Checking if F is preferred over the winner
Tactical voting possible because F is preferred over A by 2nd voter
Updated winner after tactical voting is F
Updated preference list for 2nd voter
1st preference     F
2nd preference     G
3rd preference     B
4th preference     H
5th preference     C
6th preference     E
7th preference     D
8th preference     A
9th preference     I
10th preference    J
Name: 2nd voter, dtype: object
Happiness of the voters is : 3.282377622377622
###################
Tactical voting possible because F is preferred over A by 10th voter
Updated winner after tactical voting is F
Updated preference list for 10th voter
1st preference     F
2nd preference     H
3rd preference     D
4th preference     I
5th preference     C
6th pre

In [ ]:
# Scheme 2- Voting for two -- [1,1,0,0,...,0] {Tom}

In [6]:
# Scheme 3- Anti-plurality voting (veto) -- [1,1,1,1,...,0] {Zhaolin}
##############################
### ANTI PLURALITY ##########
def AntiPluralityWinner(preference_matrix):
    sort_matrix = preference_matrix.T.sort_values('10th preference')
    unsort_lastrow = sort_matrix.sort_values('10th preference')['10th preference'].value_counts()
    sort_lastrow = unsort_lastrow.sort_index()
    possible_winner = sort_lastrow.idxmin()
    voter_preferences = preference_matrix['1st voter']
    voter_preferences = voter_preferences.sort_values()
    last_row = preference_matrix.loc['10th preference']
    for item in voter_preferences.iteritems():
        if item[1] in last_row.values:
            pass
        else:
            return item[1],sort_lastrow
    return possible_winner,sort_lastrow
    
def AntiPluralityHappiness(preference_matrix, winner):
    global_happiness = 0
    d = 0
    weight = preference_matrix.shape[0]
    for item in preference_matrix:
        voter_data = preference_matrix[item]
        voter_anti_pref = voter_data.tail(1)
        voter_last_pref = voter_anti_pref[0]
        dist = distance(voter_data,voter_last_pref,winner)
        weight_voter = weight -dist
        d= 1/(1+dist*weight_voter)
        global_happiness+= d
    print('Happiness According to Anti-Plurality is '+str(global_happiness))
    return

def AntiPluralTacticalVoting(preference_matrix, winner):
    mods = 0
    sort_matrix = preference_matrix.T.sort_values('10th preference')
    unsort_lastrow = sort_matrix.sort_values('10th preference')['10th preference'].value_counts()
    sort_lastrow = unsort_lastrow.sort_index()
    voter_preferences = preference_matrix['1st voter']
    voter_preferences = voter_preferences.sort_values()
    last_row = preference_matrix.loc['10th preference']
    min_count = sort_lastrow.min()
    possible_winners = sort_lastrow[(sort_lastrow == min_count + 1) | (sort_lastrow == min_count)]
    for item in voter_preferences.iteritems():
        if(item[1]!=winner):
            if ((item[1] not in last_row.values) or (item[1] in possible_winners)):
                mods+=CompareAntiPluralAlternative(preference_matrix,winner,item[1])
                pass
            else:
                pass
        else:
            
            pass
        pass
    return mods

def CompareAntiPluralAlternative(preference_matrix,winner,considered_winner):
    count = 0
    s = 0
    for voter in preference_matrix.iteritems():
        voter_data = voter[1]
        temp = voter_data[voter_data == winner]#GET PREFERENCE VALUE FOR WINNER
        index = temp.index
        if(index[0] != "10th preference"):
            win_position = int(index[0][0])
        else:
            win_position = 10
        temp2 = voter_data[voter_data == considered_winner]#GET PREFERENCE VALUE FOR ALTERNATIVE
        index2 = temp2.index
        if(index2.size == 0):
            print("Something wrong...voter preferences are not unique")
        if(index2[0] != "10th preference"):
            consider_position = int(index2[0][0])
        else:
            consider_position = 10
        if(consider_position<win_position):
            matrix_copy = preference_matrix.copy()
            updated_winner,updated_matrix, x=UpdateWinnerAntiPlural(matrix_copy,considered_winner,voter[0],index,winner)
            temp3 = voter_data[voter_data == updated_winner]
            index3 = temp3.index
            if(index3[0] != "10th preference"):
                updated_win_position = int(index3[0][0])
            else:
                updated_win_position = 10
            if(updated_winner!=winner and (updated_win_position < win_position) and (considered_winner==updated_winner)):
                print('Switched preference of '+ voter[0]+' between his last preference and ' +str(index[0]))
                print('Tactical voting possible because ' + updated_winner + ' is preferred over ' + winner +' by ' + voter[0])
                count+=1
                s+=1
                print('Updated Voting outcome :')
                print(x)
                print('Updated winner after tactical voting is '+str(updated_winner))
                print('Updated preference list for '+voter[0])
                print(updated_matrix[voter[0]].T)
                AntiPluralityHappiness(updated_matrix,updated_winner)
                print('#####################')
    return s
                
            
def UpdateWinnerAntiPlural(matrix,considered,voter,index_of_voter,winner):
    last_pref = matrix.at['10th preference', voter]
    matrix.at[index_of_voter,voter] = last_pref
    matrix.at['10th preference', voter] = winner
    w,x = AntiPluralityWinner(matrix)
    return w,matrix,x

#VOTING PREFERENCE- ANTI PLURALITY
Anti_Plurality_Winner,outcome = AntiPluralityWinner(preference_matrix)
print('According to Anti Plurality the winner is ' + Anti_Plurality_Winner)
print('#####################')
print('Voting Outcome')
print(outcome)
print('#####################')
AntiPluralityHappiness(preference_matrix, Anti_Plurality_Winner)
print('#####################')
mods = AntiPluralTacticalVoting(preference_matrix,Anti_Plurality_Winner)
risk = mods/preference_matrix.shape[1]
print('Risk of tactical voting according to anti plurality voting scheme for these preferences is '+str(risk))

According to Anti Plurality the winner is A
#####################
Voting Outcome
B    1
C    1
D    1
E    2
H    2
I    1
J    2
Name: 10th preference, dtype: int64
#####################
Happiness According to Anti-Plurality is 2.4175647881530233
#####################
Switched preference of 2nd voter between his last preference and 8th preference
Tactical voting possible because F is preferred over A by 2nd voter
Updated Voting outcome :
A    1
B    1
C    1
D    1
E    2
H    2
I    1
J    1
Name: 10th preference, dtype: int64
Updated winner after tactical voting is F
Updated preference list for 2nd voter
1st preference     D
2nd preference     G
3rd preference     B
4th preference     H
5th preference     C
6th preference     E
7th preference     F
8th preference     J
9th preference     I
10th preference    A
Name: 2nd voter, dtype: object
Happiness According to Anti-Plurality is 2.3223776223776222
#####################
Switched preference of 10th voter between his last preference 

In [2]:
# Data

true_pref_matrix = preference_matrix

# Scheme 4- Borda voting -- [m-1,m-2,m-2,m-3,...,0] {Chinmay}
n_candidates = true_pref_matrix.shape[0]
n_voters = true_pref_matrix.shape[1]

# Outcome calculation
def getBordaOutcome(pref_matrix, bullet_tactic=False, bullet_voter=None):
    candidate_dict = {}
    for c in range(n_candidates):
        candidate_dict[chr(65+c)] = 0

    if not bullet_tactic:
        for v in range(n_voters):
            for c in range(n_candidates):
                candidate_dict[pref_matrix.T[v,c]] += n_candidates-1-c


    else:
        for v in range(n_voters):
            if v == bullet_voter:
                candidate_dict[pref_matrix.T[v,0]] += n_candidates-1

            else:
                for c in range(n_candidates):
                    candidate_dict[pref_matrix.T[v,c]] += n_candidates-1-c

    candidate_dict.pop('nil', None)
    outcome_list = [(k,v) for k,v in candidate_dict.items()]
    outcome_list.sort(key=lambda val:val[1], reverse=True)
    outcome_list = np.array(outcome_list)[:,0]
    return outcome_list

# Individual Voter's happiness
def getVoterHappiness(v, outcome, voter_pref): # Compare the given Voter's TRUE Preference with the given Outcome
    W = list(range(1, n_candidates+1)) # weights
    W.reverse()

    d_i = 0

    for c in range(n_candidates):
        k = n_candidates - np.argwhere(outcome == voter_pref[c])[0][0] # position of the candidate in outcome (indexed form bottom)
        j = n_candidates - c # position of the candidate in the voter's pref (indexed form bottom)
        d_ij = k - j
        d_i += W[c]*d_ij
    voter_happiness = 1/(1+abs(d_i))

    return voter_happiness


# Happiness calculation
def getHappiness(outcome, true_pref_matrix):
    W = list(range(1, n_candidates+1)) # weights
    W.reverse()

    total_happiness = 0
    voter_happiness_list = []
    for v in range(n_voters):
        voter_true_pref = true_pref_matrix[:,v]

        voter_happiness = getVoterHappiness(v, outcome, voter_true_pref)
        total_happiness += voter_happiness
        voter_happiness_list.append(voter_happiness)
    return total_happiness, voter_happiness_list

################################################################################
    '''  Tactics  '''
################################################################################
def _comproTactic(v, voter_true_pref, true_pref_matrix): # Given the voter 'v' brute force over all possible compromises. Then choose the best one.
    O = getBordaOutcome(true_pref_matrix)

    # Default values for the best option
    v_h_compro_max =  getVoterHappiness(v, O, voter_true_pref)
    voter_pref_best = voter_true_pref.copy()

    for ci in range(1, n_candidates): # Start from the 2nd preference of the voter. This one will be compromised
        for cj in range(0, n_candidates):
            if cj != ci:
                voter_pref = voter_true_pref.copy()
                # Swap the values
                temp = voter_pref[ci]
                voter_pref[ci] = voter_pref[cj]
                voter_pref[cj] = temp
                # Get voter happiness
                pref_matrix = true_pref_matrix.copy()
                pref_matrix[:,v] = voter_pref.copy() #################
                O_compro = getBordaOutcome(pref_matrix)
                v_h_compro = getVoterHappiness(v, O_compro, voter_true_pref)

                if v_h_compro > v_h_compro_max: # Updating the best option
                    v_h_compro_max = v_h_compro
                    voter_pref_best = voter_pref.copy()

    modified_pref_matrix = true_pref_matrix.copy()
    modified_pref_matrix[:,v] = voter_pref_best
    O_compro = getBordaOutcome(modified_pref_matrix)
    return voter_pref_best, O_compro, v_h_compro

def _buryTactic(v, voter_true_pref, true_pref_matrix):
    O = getBordaOutcome(true_pref_matrix)

    # Default values for the best option
    v_h_bury_max =  getVoterHappiness(v, O, voter_true_pref)
    voter_pref_best = voter_true_pref.copy()

    for ci in range(0, n_candidates-1): # Stop at 2nd-last preference of the voter. This one will be buried
        for cj in range(0, n_candidates):
            if cj != ci:
                voter_pref = voter_true_pref.copy()
                # Swap the values
                temp = voter_pref[ci]
                voter_pref[ci] = voter_pref[cj]
                voter_pref[cj] = temp
                # Get voter happiness
                pref_matrix = true_pref_matrix.copy()
                pref_matrix[:,v] = voter_pref.copy()
                O_bury = getBordaOutcome(pref_matrix)
                v_h_bury = getVoterHappiness(v, O_bury, voter_true_pref)

                if v_h_bury > v_h_bury_max: # Updating the best option
                    v_h_bury_max = v_h_bury
                    voter_pref_best = voter_pref.copy()

    modified_pref_matrix = true_pref_matrix.copy()
    modified_pref_matrix[:,v] = voter_pref_best
    O_bury = getBordaOutcome(modified_pref_matrix)
    return voter_pref_best, O_bury, v_h_bury

def _bulletTactic(v, voter_true_pref, true_pref_matrix):
    #O = getBordaOutcome(true_pref_matrix) # Get non-strategic Outcome
    #v_h =  getVoterHappiness(v, O, voter_true_pref) # Get the voter's non-strategic happiness

    voter_pref_bullet = voter_true_pref[0] # Voter votes only for the most preferred candidate
    modified_pref_matrix = true_pref_matrix.copy()
    modified_pref_matrix[0,v] =  voter_pref_bullet
    #modified_pref_matrix[1:,v] = 'nil'
    O_bullet = getBordaOutcome(modified_pref_matrix, bullet_tactic=True, bullet_voter=v)
    v_h_bullet = getVoterHappiness(v, O_bullet, voter_true_pref)

    return voter_pref_bullet, O_bullet, v_h_bullet


def applyBestTactic(v, true_pref_matrix):
    O = getBordaOutcome(true_pref_matrix)
    #H, _ = getHappiness(O, true_pref_matrix)

    #Get voter happiness
    voter_true_pref = true_pref_matrix[:,v]
    v_h = getVoterHappiness(v, O, voter_true_pref)

    s_v = None

    # Compromise
    v_pref_compro, O_compro, v_h_compro = _comproTactic(v, voter_true_pref, true_pref_matrix)
    # Bury
    v_pref_bury, O_bury, v_h_bury = _buryTactic(v, voter_true_pref, true_pref_matrix)
    # Bullet
    v_pref_bullet, O_bullet, v_h_bullet = _bulletTactic(v, voter_true_pref, true_pref_matrix)

    tactic = np.argmax([v_h_compro, v_h_bury, v_h_bullet]) # Choose the tactic that results in max happiness of the uv
    if tactic == 0:
        if v_h_compro > v_h:
            s_v = ["Voter's strategic preference: {}".format(list(v_pref_compro)), "Strategic outcome: {}".format(list(O_compro)), "Voter happiness: {:.5f}".format(v_h_compro), "O_compro={} better than O={}, Voter happiness rises from {:.5f} to {:.5f}".format(list(O_compro), list(O), v_h, v_h_compro)]
    elif tactic == 1:
        if v_h_bury > v_h:
            s_v = ["Voter's strategic preference: {}".format(list(v_pref_bury)), "Strategic outcome: {}".format(list(O_bury)), "Voter happiness: {:.5f}".format(v_h_bury), "O_bury={} better than O={}, Voter happiness rises from {:.5f} to {:.5f}".format(list(O_bury), list(O), v_h, v_h_bury)]
    elif tactic == 2:
        if v_h_bullet > v_h:
            s_v = ["Voter's strategic preference: {}".format(list(v_pref_bullet)), "Strategic outcome: {}".format(list(O_bullet)), "Voter happiness: {:.5f}".format(v_h_bullet), "O_bullet={} better than O={}, Voter happiness rises from {:.5f} to {:.5f}".format(list(O_bullet), list(O), v_h, v_h_bullet)]

    return s_v



def BordaScheme(true_pref_matrix): # Main function
    # O -- Non-strategic Outcome (list)
    # H -- Non-strategic Happiness (scalar)
    # S -- STrategic voting options (dict)
    # R -- Risk (scalar)

    true_pref_matrix = true_pref_matrix.values

    O = getBordaOutcome(true_pref_matrix) # Get TRUE OUTCOME
    #print(O)
    H, voter_happiness_list = getHappiness(O, true_pref_matrix) # Get TRUE TOTAL HAPPINESS
    #print(H)

    S={}
    for i in range(n_voters): # If for all voters, if their happiness can increase, they can use tactics
        s_i = applyBestTactic(i, true_pref_matrix)
        if s_i is not None:   # If voter i's happiness increases with any strategy
            S["Voter "+str(i+1)] = s_i

    R = len(S)/n_voters

    print("NON-STRETEGIC OUTCOME: ", O)
    print("\nNON-STRETEGIC HAPPINESS: ", H)
    print("\nSTRATEGIC OPTIONS: ")
    for k,v in S.items():
      print(k, ' ---')
      for i in v:
        print(i)
      print('\n')
    print("RISK: ", R)


BordaScheme(true_pref_matrix)

NON-STRETEGIC OUTCOME:  ['A' 'B' 'G' 'F' 'C' 'D' 'H' 'E' 'I' 'J']

NON-STRETEGIC HAPPINESS:  0.233937024088

STRATEGIC OPTIONS: 
Voter 3  ---
Voter's strategic preference: ['B', 'C', 'G', 'A', 'I', 'F', 'D', 'J', 'E', 'H']
Strategic outcome: ['A', 'B', 'G', 'C', 'F', 'D', 'H', 'I', 'E', 'J']
Voter happiness: 0.05000
O_compro=['A', 'B', 'G', 'C', 'F', 'D', 'H', 'I', 'E', 'J'] better than O=['A', 'B', 'G', 'F', 'C', 'D', 'H', 'E', 'I', 'J'], Voter happiness rises from 0.04348 to 0.05000


Voter 6  ---
Voter's strategic preference: ['I', 'A', 'C', 'F', 'J', 'D', 'B', 'G', 'E', 'H']
Strategic outcome: ['A', 'B', 'G', 'F', 'C', 'D', 'H', 'I', 'E', 'J']
Voter happiness: 0.01587
O_compro=['A', 'B', 'G', 'F', 'C', 'D', 'H', 'I', 'E', 'J'] better than O=['A', 'B', 'G', 'F', 'C', 'D', 'H', 'E', 'I', 'J'], Voter happiness rises from 0.01429 to 0.01587


Voter 8  ---
Voter's strategic preference: ['C']
Strategic outcome: ['A', 'B', 'C', 'F', 'G', 'D', 'H', 'I', 'E', 'J']
Voter happiness: 0.02564
O